# Player Yearly Total

In [ ]:
import bs4 as bs
import urllib.request
import pandas as pd
import math
import csv


base_url = "http://www.footballdb.com/players/"
header_names = []
table_number = 0
player_type = ['QB', 'RB', 'WR']
player_columns = [16, 11, 13]
player_file = ["qb.csv", "rb.csv", "wr.csv"]
output_file = ['qb_yearly.csv', 'rb_yearly.csv', 'wr_yearly.csv']

def get_header(top_row):
    header_names = []
    headers = top_row.find_all('th')
    header_names.append('Name')
    header_names.append('Pos')
    header_names.append('CareerYear')
    header_names.append('Ind')
    for header in headers:
        header_names.append(header.get_text())
        #print(header_names)

    return header_names     

     
        
def get_row_data(data_items, full_name, year, player_page_ind):
    data_counter = 0
    data_values = []
    # Get the Data Items in the row
    data_values.append(full_name)
    data_values.append(player_type[POS])
    data_values.append(year)
    data_values.append(player_page_ind)
    for data_item in data_items:
        if data_item.get_text() != 'NaN' and data_item.get_text() != 'NFL Totals':
            data_values.append(data_item.get_text())
    return data_values
    
POS = 2
df = pd.read_csv(player_file[POS])


player = df

Career_Year = 0
year_data = []
counter = 0
first_time = True

for ind, value in player.iterrows():
    Career_Year = 0
    name = str(value['NAME'])
    page = int(value['PAGE'])
    if len(str(page)) < 2:
        page_num = '0' + str(page) + "/"
    else:
        page_num = str(page) + "/"

    first = name.split()[0].lower()
    first = first.replace('.', '')
    last = name.split()[1].lower()

    full_name = last + ", " + first
    full_hyphen_name = first + "-" + last
    ab_first = first[:2]
    ab_last = last[:5]
    player_page = ab_last + ab_first + page_num
    career_year = 0
    player_page_ind = full_hyphen_name + '-' + ab_last + ab_first+ page_num
    current_url = base_url + player_page_ind
    
    print(current_url)

    try:
        sauce = urllib.request.urlopen(current_url).read()
        soup = bs.BeautifulSoup(sauce, 'lxml')
        stats = soup.find_all('table')
    except:
        continue
    
    table_row = 0
    header_rows = soup.find_all('thead')
    for header_row in header_rows:
        if 'Year' in header_row.get_text():
            break
        table_row = table_row + 1

    top_row = header_rows[table_row]
    if first_time == True:
        header_val = get_header(top_row)
        first_time = False
        
    #top_row = stats[table_number]
    data_tables = stats[table_row]
    #data_rows = stat.find_all('tr')
    # Navigate Rows in Tables

    data_rows = data_tables.find_all('tr') 
    for data_row in data_rows:
        data_items = data_row.find_all('td')
        print(len(data_items))
        if len(data_items) == player_columns[POS]:
            print(data_items)
            if str(data_items).find('\xa0') == -1:
                Career_Year = Career_Year + 1
            if str(data_items).find('NFL Totals') == -1: 
                row_data = get_row_data(data_items, full_name, Career_Year, player_page_ind)
                print(row_data)
                year_data.append(row_data)
        
print("Finished with processing data")
print(len(year_data))
with open(output_file[POS], 'w') as mycsvfile:
    thedatawriter = csv.writer(mycsvfile)
    thedatawriter.writerow(header_val)
    for row in year_data:
        thedatawriter.writerow(row)
        
print('Done!')

# Player Game Totals

In [ ]:
import bs4 as bs
import urllib.request
import pandas as pd
import math
import csv



base_url = "http://www.footballdb.com/players/"
header_names = []
table_number = 0
player_type = ['QB', 'RB', 'WR']
player_columns = [16, 11, 11]
player_file = ["qb_yearly.csv", "rb_yearly.csv", "wr_yearly.csv"]
output_file = ['qb_games.csv', 'rb_games.csv', 'wr_games.csv']
#stat_type = 
stat_type = [['Pass ', 'Rush '],
            ['Rush ', 'Rec '],
            ['Rec ', 'Rush ']] 
pos_blanks = [[0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0]]
POS = 0



def get_header(top_row, i):
    headers = top_row.find_all('th')
    if i == 0:
        header_names.append('Name')
        header_names.append('Career Year')
    icounter = 0
    for header in headers:
        if i > 0:
            if icounter > 3:
                header_val = stat_type[POS][i] + header.get_text()
                header_names.append(header_val)
        else:
            if icounter > 3:
                header_val = stat_type[POS][i] + header.get_text()
                header_names.append(header_val)
            else:    
                header_names.append(header.get_text())


        icounter = icounter + 1
        #print(header_names)

    return header_names     

                
    
def get_row_data(data_items, full_name, year):
    data_counter = 0
    data_values = []
    # Get the Data Items in the row
    data_values.append(full_name)
    data_values.append(year)
    for data_item in data_items:
        data_values.append(data_item.get_text())
    return data_values

    
def get_row_data2(data_items):
    data_counter = 0
    data_values = []
    # Get the Data Items in the row
    counter = len(data_items)
    for i in range(counter):
        if i > 3:
            data_values.append(data_items[i].get_text())
    return data_values   

    
player = pd.read_csv(player_file[POS], encoding="utf-8")
#player = player.head(20)
year_data = []
counter = 0
first_time = True
row_data = []
row_data2 = []

for ind, value in player.iterrows():
    name = str(value['Name'])
    year = str(value['Year'])
    CareerYr = value['CareerYear']
    
    web_page_ind = str(value['Ind'])

    if year.isdigit() != True:
        continue
        
    current_url = base_url + web_page_ind + "gamelogs/" + year + "/"
    
    print(current_url)
    try:
        sauce = urllib.request.urlopen(current_url).read()
        soup = bs.BeautifulSoup(sauce, 'lxml')
    except:
        print("Broke for url error")
        continue

    alltables = soup.find_all('table')
    if len(alltables) == 0:
        continue
      
      
    header_rows = soup.find_all('thead')
    if len(header_rows) > 2:
        rows = 2
    else:
        rows = 1
    print(len(header_rows))
    if first_time == True:
        header_names = []
        for i in range(rows):
            print(i)
            header_names = get_header(header_rows[i], i)
            first_time = False
            print(header_names)
    
    table = alltables[0]
    data_rows = table.find_all('tr')
    if rows > 1:
        table2 = alltables[1]
        data_rows2 = table2.find_all('tr')
    row_count = len(data_rows)

    for i in range(1, row_count):
        print('in row %d of %d total rows' % (i, row_count))
        data_items = data_rows[i].find_all('td')

        if len(data_items) > 1:
            row_data = get_row_data(data_items, name, CareerYr)
        if rows > 1:
            data_items2 = data_rows2[i].find_all('td')
            row_data2 = get_row_data2(data_items2)
        else:
            row_data2 = pos_blanks[POS]
        all_row_data = row_data + row_data2
        print(all_row_data)
        year_data.append(all_row_data)
      
      

# From right script         
print("Finished with processing data")
print(len(year_data))
with open(output_file[POS], 'w') as mycsvfile:
    thedatawriter = csv.writer(mycsvfile)
    thedatawriter.writerow(header_names)
    for row in year_data:
        thedatawriter.writerow(row)

        
print('Done!')  